  
# New `opencadd` features

- New features still in a PR:
https://github.com/volkamerlab/opencadd/pull/44
- New features branch:
https://github.com/volkamerlab/opencadd/tree/add_io_klifs_subpockets

## New modules

- `opencadd.io`
- `opencadd.databases.klifs`
- `opencadd.structure.pocket`

## Showcase in this notebook how to...

- Explore kinase data (KLIFS)
- Define and visualize subpockets

## Explore kinase data (KLIFS)

In [ ]:
from opencadd.databases.klifs import setup_remote, setup_local

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

In [ ]:
remote = setup_remote()

Some of the class method options:

|                           | kinases | ligands | structures | bioactivities | interactions | pockets |
|:--------------------------| -  | -  | -  | -       | -  | -  |
| __by_kinase_klifs_id__    | x* | x* | x* |         | x  |    | 
| __by_kinase_name__        | x* | x  | x  |         |    |    |
| __by_ligand_klifs_id__    |    | x* | x  | x\* \** | x  |    |
| __by_ligand_expo_id__     |    | x  | x  | x\* \** |    |    |
| __by_structure_klifs_id__ |    |    | x* |         | x* | x* |
| __by_structure_pdb_id__   |    |    | x* |         |    |    |

 \* Direct use of KLIFS Swagger API.
 
 \** KLIFS Swagger API allows only ONE input value.


### 1. Get metadata on example ligand

In [ ]:
ligand = remote.ligands.by_ligand_expo_id("STI")  # Imatinib
ligand

### 2. Get bioactivities measured for example ligand

In [ ]:
bioactivities = remote.bioactivities.by_ligand_klifs_id(48)
bioactivities.sort_values("ligand.bioactivity_standard_value", inplace=True)
bioactivities

### 3. Get metadata on example kinase

In [ ]:
kinases = remote.kinases.by_kinase_name("ABL1")
kinases

### 4. Get structures for example kinase

In [ ]:
structures = remote.structures.by_kinase_klifs_id(392)
structures

In [ ]:
# Keep STI-bound structures
structures = structures[structures["ligand.expo_id"] == "STI"].copy()
# Sort structures by highest KLIFS quality score
structures.sort_values("structure.qualityscore", ascending=False, inplace=True)
structures

[KLIFS: a structural kinase-ligand interaction database](https://academic.oup.com/nar/article-lookup/doi/10.1093/nar/gkv1082) (Figure S4)
> Missing atoms of missing residues are not taken into account and missing residues in key positions (conserved KIII.17, conserved EαC.24, gatekeeper GK.45, DxDFG.81 and FxDFG.82 of the DFG motif) are penalized twice.

### 5. Get the IFP for an example structure

In [ ]:
# Which interactions are taken into account?
remote.interactions.interaction_types

[KLIFS: A Knowledge-Based Structural Database To Navigate Kinase–Ligand Interaction Space](https://pubs.acs.org/doi/10.1021/jm400378w)
> __For each amino acid__ in the catalytic cleft, __seven types of__ protein−ligand __interactions__ are determined. The presence of a certain type of interaction results in the type-setting of a __“1”__ in the bit-string; otherwise a __“0”__ is used to indicate the absence of the interaction.

In [ ]:
structure_ids = structures["structure.klifs_id"].to_list()
print(*structure_ids)

In [ ]:
interaction_fingerprints = remote.interactions.by_structure_klifs_id(structure_ids)
interaction_fingerprints

In [ ]:
ifps = pd.DataFrame(interaction_fingerprints["interaction.fingerprint"].apply(lambda x: list(x)).to_list())
ifps = ifps.astype('int32')
ifps.head(2)

In [ ]:
ifp_relative = (ifps.sum() / len(ifps)).to_list()
len(ifp_relative)

In [ ]:
residue_feature_matrix = pd.DataFrame(
    [ifp_relative[i:i+7] for i in range(0, len(ifp_relative), 7)], index=range(1,86)
)
residue_feature_matrix.columns = remote.interactions.interaction_types["interaction.name"].to_list()
residue_feature_matrix

In [ ]:
residue_feature_matrix.plot.bar(stacked=True, figsize=(15,5));

### 6. Get pocket for example structure

In [ ]:
pocket = remote.pockets.by_structure_klifs_id(1048)
pocket

### 7. Get coordinates

In [ ]:
remote.coordinates

#### Coordinates as `DataFrame`

In [ ]:
remote.coordinates.to_dataframe(1048, entity="complex", extension="pdb")#.dropna()

#### Coordinates saved to a _pdb_ file

In [ ]:
filepath = remote.coordinates.to_pdb(1048, ".", entity="complex")
filepath

## Define and visualize subpockets

In [ ]:
from opencadd.structure.pocket import Pocket

In [ ]:
Pocket.from_file?

In [ ]:
pocket.head(2)

In [ ]:
pocket_3d = Pocket.from_file(
    filepath, 
    pocket["residue.id"].to_list(), 
    "example kinase", 
    pocket["residue.klifs_id"].to_list()
)

### Add subpockets

In [ ]:
# Define subpockets (anchor residues, name, and color).
subpockets = {
    "anchor_residue.klifs_ids": [[16, 47, 80], [19, 24, 81], [6, 48, 75]],
    "subpocket.name": ["hinge_region", "dfg_region", "front_pocket"],
    "subpocket.color": ["magenta", "cornflowerblue", "cyan"]
}
subpockets = pd.DataFrame(subpockets)
subpockets

In [ ]:
# Map residue KLIFS IDs > residue ID.
subpockets["anchor_residue.ids"] = subpockets["anchor_residue.klifs_ids"].apply(
    lambda x: pocket[pocket["residue.klifs_id"].isin(x)]["residue.id"].to_list()
)

In [ ]:
subpockets

In [ ]:
pocket_3d.add_subpocket?

In [ ]:
subpockets

In [ ]:
for _, subpocket in subpockets.iterrows():
    pocket_3d.add_subpocket(
        subpocket["subpocket.name"], 
        subpocket["anchor_residue.ids"],
        subpocket["subpocket.color"], 
        subpocket["anchor_residue.klifs_ids"]
    )

### Check out subpockets

In [ ]:
pocket_3d.subpockets

### Check out anchor residues

In [ ]:
pocket_3d.anchor_residues

### Add regions

In [ ]:
pocket_3d.add_region?

In [ ]:
pocket.head()

In [ ]:
for (region, color), group in pocket.groupby(["residue.klifs_region_id", "residue.klifs_color"]):
    pocket_3d.add_region(
        region, 
        group["residue.id"].to_list(), 
        color, 
        group["residue.klifs_region_id"].to_list()
    )

### Check out regions

In [ ]:
pocket_3d.regions

### Visualize pocket

In [ ]:
view = pocket_3d.visualize()
view.add_representation("ball+stick", selection="STI")  # Show STI
view

In [ ]:
# Remove downloaded file
filepath.unlink()

## Thank you for your attention.

### Special thanks to Jaime for a fun code review!